# Tweet Analysis

## Load Libraries

In [36]:
import pandas as pd
import numpy as np
import psycopg2
import yaml
import psycopg2
from sqlalchemy import create_engine
import nltk
import arrow
from time import strptime
from datetime import datetime

## Functions

In [48]:
# Function to get date from "created_at field"
def genDate(date_field):
    # Tokenize by splitting on whitespace
    temp_split = date_field.split(' ')
    # First item is month, second is day, last is year. Group together into a date
    # Paste together into a string
    temp_date = temp_split[-1] + '-' + temp_split[1] + '-' + temp_split[2]
    # Convert to datetime
    temp_date = datetime.strptime(temp_date, '%Y-%b-%d')
    # Return date string
    return(temp_date)

## Connect to Database

### Read in YAML File

In [2]:
# Read in AWS creds
with open(r'/Users/samivanecky/Git/tweeter/aws.yaml') as file:
    aws = yaml.full_load(file)

### Connect to RDS

In [3]:
# Setup connection to tweeter DB
# Create engine string
connect_str = 'postgresql+psycopg2://' + aws['user'] + ':' + aws['password'] + '@' + aws['host'] + ':' + str(aws['port']) + '/'

# Create engine connection
engine = create_engine(connect_str)

c = engine.connect()
conn = c.connection

## Load Data

In [4]:
tweets = pd.read_sql('SELECT * FROM stream_tweets', engine)

## Clean Up Data

In [5]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93807 entries, 0 to 93806
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            93807 non-null  int64 
 1   _id              0 non-null      object
 2   id               93807 non-null  int64 
 3   created_at       93807 non-null  object
 4   text             93807 non-null  object
 5   source           93807 non-null  object
 6   truncated        0 non-null      object
 7   geo              0 non-null      object
 8   place            0 non-null      object
 9   quote_count      93807 non-null  int64 
 10  reply_count      93807 non-null  int64 
 11  retweet_count    93807 non-null  int64 
 12  favorite_count   93807 non-null  int64 
 13  lang             0 non-null      object
 14  is_quote_status  93807 non-null  bool  
 15  contributors     0 non-null      object
 16  search_word      93807 non-null  object
dtypes: bool(1), int64(6), object(10

In [6]:
tweets.head()

,index,_id,id,created_at,text,source,truncated,geo,place,quote_count,reply_count,retweet_count,favorite_count,lang,is_quote_status,contributors,search_word
0,0,None,1396950007434125319,Mon May 24 22:03:30 +0000 2021,RT @TheSteinLine: Memphis' OT play-in victory ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",None,None,None,0,0,0,0,None,False,None,NBA
1,1,None,1396950007593439242,Mon May 24 22:03:30 +0000 2021,RT @nchan: Spurs players setting up an in-flig...,"<a href=""http://twitter.com/download/iphone"" r...",None,None,None,0,0,0,0,None,False,None,NBA
2,2,None,1396950009808101379,Mon May 24 22:03:31 +0000 2021,RT @TheSteinLine: Memphis' OT play-in victory ...,"<a href=""http://twitter.com/download/iphone"" r...",None,None,None,0,0,0,0,None,False,None,NBA
3,3,None,1396950017747963908,Mon May 24 22:03:32 +0000 2021,RT @XXL: Gucci Mane the only rapper to double ...,"<a href=""http://twitter.com/download/iphone"" r...",None,None,None,0,0,0,0,None,False,None,NBA
4,4,None,1396950020264370178,Mon May 24 22:03:33 +0000 2021,RT @GBEshoddy: miles bridges said “feel like c...,"<a href=""http://twitter.com/download/iphone"" r...",None,None,None,0,0,0,0,None,True,None,NBA


In [49]:
# Create dates column using datefinder on "created_at"
tweets['date'] = tweets['created_at'].apply(genDate)

In [50]:
tweets

,index,_id,id,created_at,text,source,truncated,geo,place,quote_count,reply_count,retweet_count,favorite_count,lang,is_quote_status,contributors,search_word,date
0,0,None,1396950007434125319,Mon May 24 22:03:30 +0000 2021,RT @TheSteinLine: Memphis' OT play-in victory ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",None,None,None,0,0,0,0,None,False,None,NBA,2021-05-24
1,1,None,1396950007593439242,Mon May 24 22:03:30 +0000 2021,RT @nchan: Spurs players setting up an in-flig...,"<a href=""http://twitter.com/download/iphone"" r...",None,None,None,0,0,0,0,None,False,None,NBA,2021-05-24
2,2,None,1396950009808101379,Mon May 24 22:03:31 +0000 2021,RT @TheSteinLine: Memphis' OT play-in victory ...,"<a href=""http://twitter.com/download/iphone"" r...",None,None,None,0,0,0,0,None,False,None,NBA,2021-05-24
3,3,None,1396950017747963908,Mon May 24 22:03:32 +0000 2021,RT @XXL: Gucci Mane the only rapper to double ...,"<a href=""http://twitter.com/download/iphone"" r...",None,None,None,0,0,0,0,None,False,None,NBA,2021-05-24
4,4,None,1396950020264370178,Mon May 24 22:03:33 +0000 2021,RT @GBEshoddy: miles bridges said “feel like c...,"<a href=""http://twitter.com/download/iphone"" r...",None,None,None,0,0,0,0,None,True,None,NBA,2021-05-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93802,93802,None,1398647403453026305,Sat May 29 14:28:21 +0000 2021,Y'all brought the energy to @StateFarmArena la...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",None,None,None,0,0,0,0,None,False,None,NBA,2021-05-29
93803,93803,None,1398647408695840769,Sat May 29 14:28:22 +0000 2021,RT @SportsCenter: Jayson Tatum is now the only...,"<a href=""http://twitter.com/download/iphone"" r...",None,None,None,0,0,0,0,None,False,None,NBA,2021-05-29
93804,93804,None,1398647412760231941,Sat May 29 14:28:23 +0000 2021,Clippers now the favorite (-115) to win their ...,"<a href=""http://twitter.com/download/iphone"" r...",None,None,None,0,0,0,0,None,False,None,NBA,2021-05-29
93805,93805,None,1398647415817883650,Sat May 29 14:28:24 +0000 2021,RT @DrewD977ESPN: The best tweet of the night....,"<a href=""http://twitter.com/download/iphone"" r...",None,None,None,0,0,0,0,None,True,None,NBA,2021-05-29
